Tutorial: Computing with large datasets
=======================================
The Gromov-Wasserstein distance between two cells with 100 points takes about 9ms to compute on a standard desktop. The number of pairs grows quadratically with the number of cells, and so the total runtime can become large.

For large datasets we provide two tools to reduce the necessary computation, as well as a hybrid of these.

In <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) the author establishes several lower bounds for the Gromov-Wasserstein distance. We have implemented one of the fastest, which we name "SLB2"<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2). For the purposes of clustering analysis it may be enough to know the precise Gromov-Wasserstein distance only for cells which are close to each other in the morphology space, because most clustering algorithms are insensitive to the precise distance between points which are very far apart. Similarly, many dimensionality reduction techniques only respect local distances but do not preserve global structure of the space, and so it is not crucial to know the exact values between disparate cells.

In [2]:
import os
cwd= os.getcwd()
icdm_file=os.path.join(cwd,"icdms/swc_100pts_eu_4_17_2023.csv")

If the SLB2 distance between all cells are known, there is a straightforward algorithm<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3) to compute the $k$ nearest neighbors of any given cell $c_0$ under the Gromov-Wasserstein metric. If $k$ is chosen sufficiently large, this is enough to understand the local structure of the morphology space and cluster it.

The second tool we provide is an implementation of the quantized Gromov-Wasserstein distance proposed by Chowdhury, Miller and Needham[^4]. Given cells $X$ and $Y$, the quantized Gromov-Wasserstein distance is given by partitioning the points of $X$ and $Y$ into clusters, computing the optimal Gromov-Wasserstein coupling matrix between the subspaces formed by the medoids of each cluster, and extending this to a global transport plan by pairing points within paired clusters by their distance from the medoid. This approximation gives an acceptable tradeoff between precision and computation time.

We combine these two tools in an integrated analysis method which tries to reduce

<a name="cite_note-1"></a>1.[^](#cite_ref-1) Mémoli, F. P. [Gromov–Wasserstein Distances and the Metric Approach to Object Matching.](https://dblp.uni-trier.de/db/journals/focm/focm11.html#Memoli11) Found Comput Math (2011) 11, 417–487.

<a name="cite_note-2"></a>2.[^](#cite_ref-2) Specifically we have implemented the expression which appears on the right hand side of the first inequality of Corollary 6.2 on page 462, for p = 2. This expression is not directly named in the paper. This

<a name="cite_note-3"></a>3.[^](#cite_ref-3) A simple algorithm for computing the nearest neighbors of a cell in the Gromov-Wasserstein morphology space if the SLB2 distance is known is as follows:
1. First, sort all other cells by their SLB2 distance from $c_0$, as $c_1, c_2, c_3,\dots$.
2. Next, compute the Gromov-Wasserstein distance $GW(c_0,c_j)$, as $j = 1, 2, 3,\dots$. Write $e^k_j$ for the $k$-th element of the set $GW(c_0,c_1),GW(c_0,c_2),\dots,GW(c_0,c_j)$ when these are ordered from least to greatest. ($e^k_j$ is only defined when $k \leq j$). Continue computing $GW(c_0,c_j)$ until $j$ reaches a value $\ell$ such that for all $i> \ell$, $SLB(c_0,c_i) > e^k_\ell$. Because SLB is a lower bound, at this point, the $k$ nearest neighbors of $c_0$ are contained in the set $\{c_1,\dots, c_\ell\}$.

[^4] Chowdhury, S., Miller, D., Needham, T. (2021). Quantized Gromov-Wasserstein. In: Oliver, N., Pérez-Cruz, F., Kramer, S., Read, J., Lozano, J.A. (eds) Machine Learning and Knowledge Discovery in Databases. Research Track. ECML PKDD 2021. Lecture Notes in Computer Science(), vol 12977. Springer, Cham. https://doi.org/10.1007/978-3-030-86523-8_49



contribute to the neuronal plasticity in the C. elegans. This example utilizes a dataset consisting of 799 3D neuronal reconstructions of the C.elegans DVB neuron across various mutant and control strains during days 1 to 5 of adulthood. The dataset can be downloaded from the following
[folder](https://www.dropbox.com/scl/fo/y7axeardkwyn1d6j97dqr/h?dl=0&rlkey=4b65w4tc93e778rql72d275ym). The DVB neuron is an excitatory GABAergic motor
interneuron located in the dorso-rectal ganglion of the worm, and is known to undergo post-developmental neurite outgrowth in males. This outgrowth 
alters the neuron's morphology and synaptic connectivity, contributing to
changes in the spicule protraction step of male mating behavior. More information about this 
dataset can be found at:

\- Hart, M. P. & Hobert, O. [Neurexin controls plasticity of a mature, sexually dimorphic neuron.](https://www.nature.com/articles/nature25192) Nature 553, 165-170, (2018).

\- Govek, K. W. et al. [Analysis and integration of single-cell morphological data using metric geometry.](https://www.biorxiv.org/content/10.1101/2022.05.19.492525v3) (2022). DOI: 10.1101/2022.05.19.492525 (bioRxiv).

To begin our analysis, we calculate the Gromov-Wasserstein distance between each pair of cells. For the sake of time, here we just sample 50 points per cell. This computation typically requires 20-30 minutes to complete on a standard desktop computer. A larger number of sampled points would offer better results, but would also increase the computing time.